In [ ]:
# Data Processing from base data to dataset for inflation adjustment factor

In [ ]:
import pandas as pd

df_clean = pd.read_json("../../public/data/base_data.json")

# drop needless columns

In [ ]:
# Export to JSON
# df_clean.to_json('../../public/data/inflation_data.json')